In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder
import keras
from keras import layers
from sklearn.preprocessing import Normalizer

In [18]:
df = pd.read_csv('Airplane.csv')
df.head()

,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied


In [19]:
df.shape

(103904, 25)

In [20]:
columns_to_convert = ['Gender', 'Customer Type', 'Type of Travel',
                      'Class', 'satisfaction']
label_encoder = LabelEncoder()
for column in columns_to_convert:
    df[column] = label_encoder.fit_transform(df[column])


In [21]:

random_indices = np.random.choice(df.shape[0] - 1, 9000, replace=False)
selected_data = df.iloc[random_indices, :]

print(selected_data)

       Unnamed: 0      id  Gender  Customer Type  Age  Type of Travel  Class  \
60216       60216    4368       1              0   60               0      0   
8483         8483   81467       0              0   67               0      0   
57067       57067   28668       1              0   43               0      2   
95298       95298  128006       0              0   59               1      1   
59140       59140    3591       1              0   58               0      0   
...           ...     ...     ...            ...  ...             ...    ...   
85494       85494  127735       0              0   49               0      0   
15658       15658   14171       1              0   37               0      0   
3586         3586   79928       1              0   51               0      0   
15091       15091   95590       0              0   26               0      0   
45621       45621    8344       1              0   36               1      2   

       Flight Distance  Inflight wifi s

In [22]:
selected_data.shape

(9000, 25)

In [23]:
X = selected_data.iloc[:, :-1]
y = selected_data.iloc[:, -1]

In [24]:
imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
X_imputed = pd.DataFrame(imputer.fit_transform(X), columns = X.columns)

In [25]:
normalizer = Normalizer()
normalizer.fit(X_imputed)
X_norm = normalizer.transform(X_imputed)
print(X_norm)

[[9.97348278e-01 7.23465072e-02 1.65628451e-05 ... 4.96885352e-05
  0.00000000e+00 0.00000000e+00]
 [1.03503600e-01 9.94003044e-01 0.00000000e+00 ... 4.88051871e-05
  1.22012968e-05 6.10064838e-05]
 [8.92943661e-01 4.48576390e-01 1.56472858e-05 ... 7.82364292e-05
  0.00000000e+00 0.00000000e+00]
 ...
 [4.48117422e-02 9.98804498e-01 1.24963029e-05 ... 6.24815145e-05
  2.12437149e-04 2.49926058e-05]
 [1.55829585e-01 9.87061830e-01 0.00000000e+00 ... 4.13039787e-05
  5.16299733e-05 8.26079573e-05]
 [9.83582151e-01 1.79895431e-01 2.15598551e-05 ... 1.07799276e-04
  0.00000000e+00 0.00000000e+00]]


In [26]:
X_train, X_test, y_train, y_test = train_test_split(
    X_norm, y, test_size=0.2, random_state=42)


In [31]:
model = keras.Sequential()
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))  # Dropout layer to prevent overfitting
model.add(layers.Dense(128, activation='ELU'))
model.add(layers.Dropout(0.5))
# model.add(layers.Dense(32, activation='ELU'))
# model.add(layers.Dropout(0.5))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dropout(0.5))
# model.add(layers.Dense(128, activation='relu'))
# model.add(layers.Dropout(0.55))
# model.add(layers.Dense(64, activation='relu'))
# model.add(layers.Dropout(0.55))
# model.add(layers.Dense(128, activation='relu'))
# model.add(layers.Dropout(0.35))
model.add(layers.Dense(1, activation='sigmoid'))

In [32]:
model.compile(loss = 'binary_crossentropy',
              optimizer=keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [33]:
model.fit(X_train, y_train, epochs=50, batch_size=40,
          verbose=1, validation_split=0.2)

Epoch 1/50
144/144 [==============================] - 1s 3ms/step - loss: 0.6914 - accuracy: 0.5481 - val_loss: 0.6878 - val_accuracy: 0.5757
Epoch 2/50
144/144 [==============================] - 0s 2ms/step - loss: 0.6897 - accuracy: 0.5531 - val_loss: 0.6845 - val_accuracy: 0.5757
Epoch 3/50
144/144 [==============================] - 0s 2ms/step - loss: 0.6870 - accuracy: 0.5637 - val_loss: 0.6869 - val_accuracy: 0.5757
Epoch 4/50
144/144 [==============================] - 0s 2ms/step - loss: 0.6868 - accuracy: 0.5616 - val_loss: 0.6825 - val_accuracy: 0.5757
Epoch 5/50
144/144 [==============================] - 0s 2ms/step - loss: 0.6866 - accuracy: 0.5590 - val_loss: 0.6845 - val_accuracy: 0.5757
Epoch 6/50
144/144 [==============================] - 0s 3ms/step - loss: 0.6868 - accuracy: 0.5625 - val_loss: 0.6845 - val_accuracy: 0.5757
Epoch 7/50
144/144 [==============================] - 0s 2ms/step - loss: 0.6864 - accuracy: 0.5628 - val_loss: 0.6832 - val_accuracy: 0.5757
Epoch 

In [34]:
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

57/57 [==============================] - 0s 1ms/step
Accuracy: 64.94%
